# Split large dtag files into individual trials and filter the small files

This notebook:
- splits the long Dtag wav files into smaller ones for individual trials
- filter the smaller individual trial wav files

Content of this notebook is identical to `click_tool_notebooks/split_dtag_into_trials_wj.ipynb` in the old repo.

In [1]:
from pathlib import Path
import datetime

import pandas as pd

import soundfile as sf
from scipy import signal
from scipy.io import wavfile

## Set up paths

In [2]:
analysis_path = Path("../data_processed/")
raw_path = Path("../data_raw/")

In [3]:
split_path = analysis_path / "dtag/dtag_split_wav/"
filtered_path = analysis_path / 'dtag/dtag_split_wav_filtered/'
filtered_bpf_path = analysis_path / 'dtag/dtag_split_wav_filtered_bpf/'

for p in [split_path, filtered_path, filtered_bpf_path]:
    if not p.exists():
        p.mkdir()

## Load trial info

In [4]:
df_main = pd.read_csv(
    analysis_path / "all_info_csv/main_info_append_09.csv",
    index_col=0
)

## Split wav files

In [5]:
# get fs from first file
_, fs_dtag = sf.read(raw_path / df_main.loc[0, 'dtag_wav_file'], start=0, stop=1)
fs_dtag

576000

In [6]:
def get_smpl_num(t_str, smpl_freq):
    tt = datetime.datetime.strptime(t_str, '%M:%S.%f')
    return int(smpl_freq * (tt.minute * 60 + tt.second + tt.microsecond * 1e-6))

In [7]:
# Loop through all trials
for trial_idx in df_main.index:
    
    print('----------------------------------------------')
    print(f'Processing trial {trial_idx}')

    ts = df_main.loc[trial_idx]
    
    # Split wav file
    start_smpl = get_smpl_num(ts['dtag_trial_start_time'], fs_dtag)
    end_smpl = get_smpl_num(ts['dtag_trial_end_time'], fs_dtag)
    sig, sig_fs = sf.read(
        raw_path / ts['dtag_wav_file'], start=start_smpl, stop=end_smpl
    )
    split_wav_fname = split_path.joinpath(
        f"t{int(trial_idx):03d}"
        f"_{ts['DATE']}"
        f"_s{ts['SESSION']}"
        f"_t{ts['TRIAL']:02d}_dtag.wav"
    )
    wavfile.write(split_wav_fname, fs_dtag, sig)
    print("-- split wav: \n   " + str(split_wav_fname))
    
    # Filter wave file
    sos = signal.butter(N=40, Wn=50e3, btype='highpass',
                       output='sos', fs=fs_dtag)
    sig_filt = signal.sosfiltfilt(sos, sig)
    filtered_wav_fname = filtered_path / (split_wav_fname.stem + '_filtered.wav')
    wavfile.write(filtered_wav_fname, fs_dtag, sig_filt)
    print("-- filtered wav: \n   " + str(filtered_wav_fname))
    
    # Filter signal to only forward click band: 110-145 kHz
    sos = signal.butter(N=40, Wn=[110e3, 145e3], btype='band',
                        output='sos', fs=fs_dtag)
    sig_bpf = signal.sosfiltfilt(sos, sig)    
    bpf_wav_fname = filtered_bpf_path / (filtered_wav_fname.stem + '_bpf.wav')
    wavfile.write(bpf_wav_fname, fs_dtag, sig_bpf)
    print("-- bandpass filtered wav: \n   " + str(bpf_wav_fname))
    

----------------------------------------------
Processing trial 0
-- split wav: 
   ../data_processed/dtag/dtag_split_wav/t000_20190625_s1_t00_dtag.wav
-- filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered/t000_20190625_s1_t00_dtag_filtered.wav
-- bandpass filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered_bpf/t000_20190625_s1_t00_dtag_filtered_bpf.wav
----------------------------------------------
Processing trial 1
-- split wav: 
   ../data_processed/dtag/dtag_split_wav/t001_20190625_s1_t00_dtag.wav
-- filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered/t001_20190625_s1_t00_dtag_filtered.wav
-- bandpass filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered_bpf/t001_20190625_s1_t00_dtag_filtered_bpf.wav
----------------------------------------------
Processing trial 2
-- split wav: 
   ../data_processed/dtag/dtag_split_wav/t002_20190625_s1_t01_dtag.wav
-- filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered/t002_20190625_

-- filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered/t021_20190625_s3_t06_dtag_filtered.wav
-- bandpass filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered_bpf/t021_20190625_s3_t06_dtag_filtered_bpf.wav
----------------------------------------------
Processing trial 22
-- split wav: 
   ../data_processed/dtag/dtag_split_wav/t022_20190625_s3_t07_dtag.wav
-- filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered/t022_20190625_s3_t07_dtag_filtered.wav
-- bandpass filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered_bpf/t022_20190625_s3_t07_dtag_filtered_bpf.wav
----------------------------------------------
Processing trial 23
-- split wav: 
   ../data_processed/dtag/dtag_split_wav/t023_20190625_s3_t08_dtag.wav
-- filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered/t023_20190625_s3_t08_dtag_filtered.wav
-- bandpass filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered_bpf/t023_20190625_s3_t08_dtag_filtered_bpf.wav
-

-- filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered/t043_20190626_s2_t08_dtag_filtered.wav
-- bandpass filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered_bpf/t043_20190626_s2_t08_dtag_filtered_bpf.wav
----------------------------------------------
Processing trial 44
-- split wav: 
   ../data_processed/dtag/dtag_split_wav/t044_20190626_s2_t09_dtag.wav
-- filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered/t044_20190626_s2_t09_dtag_filtered.wav
-- bandpass filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered_bpf/t044_20190626_s2_t09_dtag_filtered_bpf.wav
----------------------------------------------
Processing trial 45
-- split wav: 
   ../data_processed/dtag/dtag_split_wav/t045_20190626_s2_t10_dtag.wav
-- filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered/t045_20190626_s2_t10_dtag_filtered.wav
-- bandpass filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered_bpf/t045_20190626_s2_t10_dtag_filtered_bpf.wav
-

-- filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered/t065_20190627_s1_t10_dtag_filtered.wav
-- bandpass filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered_bpf/t065_20190627_s1_t10_dtag_filtered_bpf.wav
----------------------------------------------
Processing trial 66
-- split wav: 
   ../data_processed/dtag/dtag_split_wav/t066_20190627_s1_t11_dtag.wav
-- filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered/t066_20190627_s1_t11_dtag_filtered.wav
-- bandpass filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered_bpf/t066_20190627_s1_t11_dtag_filtered_bpf.wav
----------------------------------------------
Processing trial 67
-- split wav: 
   ../data_processed/dtag/dtag_split_wav/t067_20190627_s2_t01_dtag.wav
-- filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered/t067_20190627_s2_t01_dtag_filtered.wav
-- bandpass filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered_bpf/t067_20190627_s2_t01_dtag_filtered_bpf.wav
-

-- filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered/t087_20190627_s3_t10_dtag_filtered.wav
-- bandpass filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered_bpf/t087_20190627_s3_t10_dtag_filtered_bpf.wav
----------------------------------------------
Processing trial 88
-- split wav: 
   ../data_processed/dtag/dtag_split_wav/t088_20190627_s3_t11_dtag.wav
-- filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered/t088_20190627_s3_t11_dtag_filtered.wav
-- bandpass filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered_bpf/t088_20190627_s3_t11_dtag_filtered_bpf.wav
----------------------------------------------
Processing trial 89
-- split wav: 
   ../data_processed/dtag/dtag_split_wav/t089_20190628_s1_t01_dtag.wav
-- filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered/t089_20190628_s1_t01_dtag_filtered.wav
-- bandpass filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered_bpf/t089_20190628_s1_t01_dtag_filtered_bpf.wav
-

...

-- filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered/t153_20190702_s1_t02_dtag_filtered.wav
-- bandpass filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered_bpf/t153_20190702_s1_t02_dtag_filtered_bpf.wav
----------------------------------------------
Processing trial 154
-- split wav: 
   ../data_processed/dtag/dtag_split_wav/t154_20190702_s1_t03_dtag.wav
-- filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered/t154_20190702_s1_t03_dtag_filtered.wav
-- bandpass filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered_bpf/t154_20190702_s1_t03_dtag_filtered_bpf.wav
----------------------------------------------
Processing trial 155
-- split wav: 
   ../data_processed/dtag/dtag_split_wav/t155_20190702_s1_t04_dtag.wav
-- filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered/t155_20190702_s1_t04_dtag_filtered.wav
-- bandpass filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered_bpf/t155_20190702_s1_t04_dtag_filtered_bpf.wav

-- filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered/t175_20190702_s3_t03_dtag_filtered.wav
-- bandpass filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered_bpf/t175_20190702_s3_t03_dtag_filtered_bpf.wav
----------------------------------------------
Processing trial 176
-- split wav: 
   ../data_processed/dtag/dtag_split_wav/t176_20190702_s3_t04_dtag.wav
-- filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered/t176_20190702_s3_t04_dtag_filtered.wav
-- bandpass filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered_bpf/t176_20190702_s3_t04_dtag_filtered_bpf.wav
----------------------------------------------
Processing trial 177
-- split wav: 
   ../data_processed/dtag/dtag_split_wav/t177_20190702_s3_t05_dtag.wav
-- filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered/t177_20190702_s3_t05_dtag_filtered.wav
-- bandpass filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered_bpf/t177_20190702_s3_t05_dtag_filtered_bpf.wav

-- split wav: 
   ../data_processed/dtag/dtag_split_wav/t197_20190703_s2_t05_dtag.wav
-- filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered/t197_20190703_s2_t05_dtag_filtered.wav
-- bandpass filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered_bpf/t197_20190703_s2_t05_dtag_filtered_bpf.wav
----------------------------------------------
Processing trial 198
-- split wav: 
   ../data_processed/dtag/dtag_split_wav/t198_20190703_s2_t06_dtag.wav
-- filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered/t198_20190703_s2_t06_dtag_filtered.wav
-- bandpass filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered_bpf/t198_20190703_s2_t06_dtag_filtered_bpf.wav
----------------------------------------------
Processing trial 199
-- split wav: 
   ../data_processed/dtag/dtag_split_wav/t199_20190703_s2_t07_dtag.wav
-- filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered/t199_20190703_s2_t07_dtag_filtered.wav
-- bandpass filtered wav: 
   ../data

-- bandpass filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered_bpf/t218_20190704_s1_t05_dtag_filtered_bpf.wav
----------------------------------------------
Processing trial 219
-- split wav: 
   ../data_processed/dtag/dtag_split_wav/t219_20190704_s1_t06_dtag.wav
-- filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered/t219_20190704_s1_t06_dtag_filtered.wav
-- bandpass filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered_bpf/t219_20190704_s1_t06_dtag_filtered_bpf.wav
----------------------------------------------
Processing trial 220
-- split wav: 
   ../data_processed/dtag/dtag_split_wav/t220_20190704_s1_t07_dtag.wav
-- filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered/t220_20190704_s1_t07_dtag_filtered.wav
-- bandpass filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered_bpf/t220_20190704_s1_t07_dtag_filtered_bpf.wav
----------------------------------------------
Processing trial 221
-- split wav: 
   ../data_processed/dt

-- filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered/t240_20190704_s3_t06_dtag_filtered.wav
-- bandpass filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered_bpf/t240_20190704_s3_t06_dtag_filtered_bpf.wav
----------------------------------------------
Processing trial 241
-- split wav: 
   ../data_processed/dtag/dtag_split_wav/t241_20190704_s3_t07_dtag.wav
-- filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered/t241_20190704_s3_t07_dtag_filtered.wav
-- bandpass filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered_bpf/t241_20190704_s3_t07_dtag_filtered_bpf.wav
----------------------------------------------
Processing trial 242
-- split wav: 
   ../data_processed/dtag/dtag_split_wav/t242_20190704_s3_t08_dtag.wav
-- filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered/t242_20190704_s3_t08_dtag_filtered.wav
-- bandpass filtered wav: 
   ../data_processed/dtag/dtag_split_wav_filtered_bpf/t242_20190704_s3_t08_dtag_filtered_bpf.wav